In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt


random.seed(5) #for reproducability
chunk_size = 256 #configure as needed
columns_for_train = ['1.0 std','2.5 std', '10 std'] #choose what to train on

In [15]:
def Linear_Scale(dataframe, column): #normal absolute max normalization
    dataframe[column] = (dataframe[column] - dataframe[column].min()) / (dataframe[column].max() - dataframe[column].min())
    
def Z_norm(dataframe, column): #see z-score normalization, better for outlier detection
    dataframe[column] = (dataframe[column] - dataframe[column].mean()) / dataframe[column].std()

def dfsplitter(dflist): #receives iterable list of dataframe 'chunks', and returns three separate lists containing 70%, 20% and final 10% of the chunks
    percent70 = int(len(dflist) * 0.7) #number of chunks = length of list, convert to int to round down any decimals
    percent20 = int(len(dflist) * 0.2)
    
    first70 = dflist[0 : percent70] #recieve items in big list from 0 to 70%
    next20 = dflist[percent70 : percent70 + percent20] #from 70 to 70+20 = 90%
    last10 = dflist[percent70 + percent20 : ] #whatever is left. will not always be exactly 10% due to earlier rounding
    return(first70, next20, last10) 

def combine_and_shuffle(list1, list2): #take two lists of dataframes, combine them, shuffle the new list, and concatenate into a single dataframe
    dflist = list1 + list2
    random.shuffle(dflist)
    return pd.concat(dflist, ignore_index=True)

def df_to_Xy(dataframe): #converts a given dataframe to X and y numpy arrays, used for training, testing and validation
    X, y = dataframe.loc[:, columns_for_train], dataframe.loc[:, 'Fire'] #X is for train, y is boolean fire values
    
    X, y = X.to_numpy(), y.to_numpy() #convert each to numpy arrays
    
    y = y.flatten() #reshaping each
    X = X.reshape((X.shape[0], X.shape[1], 1)) #maintain rows, number of columns, and add third dimenion (1 item each)
    return(X, y)

In [19]:
random.seed(5) #reproducability

df = pd.read_csv('processed.csv', chunksize=chunk_size, iterator=True) #df is now a TextFileReader dtype
allchunks = [pd.DataFrame(chunk) for chunk in df]
random.shuffle(allchunks) #a list of chunk-sized dataframes, randomized order

empty = pd.read_csv('empty.csv', chunksize=chunk_size, iterator=True) #TextFileReader again
allempties = []

for chunk in empty:
    chunk.loc[chunk.sample(frac=0.004, random_state=2).index, columns_for_train] = 5996 #take a sample of each chunk and convert all values in the row
    for column in columns_for_train: #for each column in each chunk:
        Z_norm(chunk, column)
        Linear_Scale(chunk, column)
    allempties.append(pd.DataFrame(chunk)) #add the chunk as a dataframe to a list, resulting in another list of dataframes


first70, next20, last10 = dfsplitter(allchunks) #split first df list into training, validation and testing sizes
empty70, empty20, empty10 = dfsplitter(allempties) #same thing for list of error values ('empty' as in no fire spikes)

final70 = combine_and_shuffle(first70, empty70) #take two df lists, combine, shuffle, concatenate into a DataFrame
final20 = combine_and_shuffle(next20, empty20)
final10 = combine_and_shuffle(last10, empty10)
#plt.plot(final70.index[:], final70['1.0 std'][:], final70['Fire'][:]) #for testing

In [21]:
X_train, y_train = df_to_Xy(final70) #take train-sized DataFrame, pull data and boolean values, and convert to X and y
X_val, y_val = df_to_Xy(final20)
X_test, y_test = df_to_Xy(final10)

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((26112, 3, 1), (26112,), (7174, 3, 1), (7174,), (4001, 3, 1), (4001,))

In [13]:
model = keras.Sequential()
model.add(keras.Input(shape=(3, 1))) #no specific amount of timesteps, 3 pixels in each row/timestep
model.add(layers.GRU(512, return_sequences=True, activation='relu')) #returns the output of each timestep so we can stack multiple RNN layers
model.add(layers.GRU(256, activation='relu')) #no return sequences so that normal dense layers can proceed it
model.add(layers.Dense(2))

#could do LSTM, GRU, or RNN and can change by literally just swapping them out

print(model.summary())

#RNNs typically use tanh instead of relu

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), #fromlogits is true since we don't have softmax activation in final dense layer
    optimizer=keras.optimizers.Adam(learning_rate=0.001), #dont make learning rate too high or will result in overfitment of training data
    metrics=['accuracy']
)

model.fit(X_train, y_train, batch_size=32, validation_data=(X_val, y_val),epochs=10, verbose=1)

model.evaluate(X_test, y_test, batch_size=32, verbose=1)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 3, 512)         │       791,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 256)            │       591,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,382,914 (5.28 MB)

 Trainable params: 1,382,914 (5.28 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9756 - loss: 0.1082 - val_accuracy: 0.9869 - val_loss: 0.0407
Epoch 2/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9798 - loss: 0.0465 - val_accuracy: 0.9869 - val_loss: 0.0308
Epoch 3/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.9848 - loss: 0.0377 - val_accuracy: 0.9873 - val_loss: 0.0306
Epoch 4/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.9858 - loss: 0.0335 - val_accuracy: 0.9884 - val_loss: 0.0249
Epoch 5/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9866 - loss: 0.0316 - val_accuracy: 0.9912 - val_loss: 0.0205
Epoch 6/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.9880 - loss: 0.0303 - val_accuracy: 0.9907 - val_loss: 0.0232
Epoch 7/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9885 - loss: 0.0282 - val_accuracy: 0.9932 - val_loss: 0.0247
Epoch 8/10
816/816 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.9878 - loss: 0.02

[0.034934915602207184, 0.9880030155181885]